In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import loadmat
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

In [2]:
class SVHN_dataset(Dataset):

    # def __init__(self, data,transform):
    def __init__(self, data):  
        # Transform data to Torch Tensors
        self.images = torch.tensor(data['X']).permute([3,2,0,1])
        self.labels = torch.tensor(data['y'])
        self.size = self.labels.shape[0]

        # replace label 10 with label 0
        self.labels[self.labels==10] = 0
        # convert to float and normalize images to 0..1 range
        self.images = torch.FloatTensor(self.images/255.)
        # self.transform = transform

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [3]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(3,6,(5,5),padding=2,stride=1),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6,12,(3,3),padding=1,stride=1),
        nn.BatchNorm2d(12),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(12,24,(3,3),padding=1,stride=1),
        nn.BatchNorm2d(24),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc1 = nn.Linear(24*4*4, 10)
    # self.fc1 = nn.Linear(in_features=24*4*4, out_features=120)
    # self.fc2 = nn.Linear(in_features=120, out_features=60)
    # self.out = nn.Linear(in_features=60, out_features=10)
  

  def forward(self, x):
    out = self.layer1(x)
    # print(out.shape)

    out = self.layer2(out)
    # print(out.shape)

    out = self.layer3(out)
    # print(out.shape)

    out = out.reshape(-1, 24 * 4 * 4)
    out = self.fc1(out)
    # out = F.relu(out)

    # out = self.fc2(out)
    # out = F.relu(out)

    # out = self.out(out)
    # out = F.softmax(out)
    # print(out.shape)

    return out

In [4]:
# initialize weights of CNN layers
def init_weights(m):
  mean = 0.0
  std = 0.001
  if isinstance(m, nn.Conv2d):
    m.weight.data.normal_(mean,std)
    if m.bias is not None:
      nn.init.constant_(m.bias.data, 0)
  elif isinstance(m, nn.BatchNorm2d):
    m.weight.data.normal_(mean,std)
    if m.bias is not None:
      nn.init.constant_(m.bias.data, 0)
  elif isinstance(m, nn.Linear):
    m.weight.data.normal_(mean,std)
    if m.bias is not None:
      nn.init.constant_(m.bias.data, 0)

In [ ]:
# transform = transforms.Compose(
#       [
#         transforms.ToTensor(),
#         transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
#         ]
#     )

In [5]:
train = loadmat('train_32x32.mat')
test = loadmat('test_32x32.mat')

In [6]:
print(train['X'].shape)

(32, 32, 3, 73257)


In [7]:
# train_SVHNdataset = SVHN_dataset(data = train, transform = transform)
# test_SVHNdataset = SVHN_dataset(data = test, transform = transform)
train_SVHNdataset = SVHN_dataset(data = train)
test_SVHNdataset = SVHN_dataset(data = test)

In [8]:
params = {'batch_size': 512,
          'shuffle': True,
          'num_workers': 2,
          'pin_memory':True}

In [9]:
trainSVHN_loader = DataLoader(train_SVHNdataset, **params)
testSVHN_loader = DataLoader(test_SVHNdataset, **params)

In [10]:
model = CNN()
model.apply(init_weights)
device = torch.device("cuda:0" if torch.cuda.is_available() else cpu)
print(device)

cuda:0


In [11]:
model.to(device)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=384, out_features=10, bias=True)
)

In [12]:
learning_rate = 0.001
lossfunc = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [13]:
num_epochs = 50
# loss_values = list()
for epoch in range(num_epochs):
    # Training
    training_loss = 0
    model.train()
    for X_train, y_train in trainSVHN_loader:
      # Transfer to GPU
      X_train_tensor = torch.tensor(X_train, dtype = torch.float)
      y_train_tensor = torch.tensor(y_train, dtype = torch.long)

      images, labels = Variable(X_train_tensor).to(device), Variable(y_train_tensor).to(device)

      # model computation
      outputs = model(images)
      target = labels.squeeze()
      loss = lossfunc(outputs, target)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      training_loss += loss.item()
    training_loss /= len(trainSVHN_loader)
    print('Epoch -',epoch, 'training loss - ', training_loss)
print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, cei

Epoch - 0 training loss -  2.2212345004081726
Epoch - 1 training loss -  1.4563380016220941
Epoch - 2 training loss -  0.8480565663841035
Epoch - 3 training loss -  0.7231536867717901
Epoch - 4 training loss -  0.6686514860226048
Epoch - 5 training loss -  0.6316460789077811
Epoch - 6 training loss -  0.6097860530846648
Epoch - 7 training loss -  0.5863562636077404
Epoch - 8 training loss -  0.5715276704480251
Epoch - 9 training loss -  0.5594563221351968
Epoch - 10 training loss -  0.5470230927069982
Epoch - 11 training loss -  0.540867736356126
Epoch - 12 training loss -  0.5263313996709056
Epoch - 13 training loss -  0.5206435558696588
Epoch - 14 training loss -  0.5153970008509027
Epoch - 15 training loss -  0.5099788310213221
Epoch - 16 training loss -  0.5037676617503166
Epoch - 17 training loss -  0.496163013494677
Epoch - 18 training loss -  0.4925959629731046
Epoch - 19 training loss -  0.4885523182650407
Epoch - 20 training loss -  0.48774432287447983
Epoch - 21 training loss

In [14]:
model.eval()
with torch.no_grad():
  correct = 0
  batches = 0
  for X_test, y_test in testSVHN_loader:
      X_test_tensor = torch.tensor(X_test, dtype = torch.float)
      y_test_tensor = torch.tensor(y_test, dtype = torch.long)
      images, labels = Variable(X_test_tensor).to(device), Variable(y_test_tensor).to(device)
      outputs = model(images)
      target = labels.squeeze()
      _, predicted = torch.max(outputs.data, 1)
      batches += labels.size(0)
      correct += (predicted == target).sum().item()

  print('Test Accuracy of the model: {} %'.format(100 * correct / batches))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Test Accuracy of the model: 84.53441917639827 %
